# Data drift dashboard in jupyter notebook

## Imports

In [1]:
pip install evidently

Note: you may need to restart the kernel to use updated packages.


In [2]:
import io
import numpy as np
import os
import pandas as pd
from pathlib import Path
import requests
import zipfile

from datetime import datetime
from sklearn import datasets, ensemble

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import TargetDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset.regression_performance import RegressionPreset 

from evidently.metrics import (
    RegressionQualityMetric,
    RegressionPredictedVsActualScatter,
    RegressionPredictedVsActualPlot,
    RegressionErrorPlot,
    RegressionAbsPercentageErrorPlot,
    RegressionErrorDistribution,
    RegressionErrorNormality,
    RegressionTopErrorMetric,
    RegressionErrorBiasTable,
    
    DatasetSummaryMetric,
    ColumnSummaryMetric,
    DatasetMissingValuesMetric,
    DatasetCorrelationsMetric
)

## Bicycle Demand Data

### Download and extract (unzip) data

This step automatically downloads the bike dataset from UCI. This version is slightly different from the dataset used in Kaggle competition. If you want the example to be identical to the one in the Evidently blog "How to break a model in 20 days", you can manually download the dataset from Kaggle: https://www.kaggle.com/c/bike-sharing-demand/data 

And add this code:

raw_data['mnth'] = raw_data.index.map(lambda x : x.month)

raw_data['hr'] = raw_data.index.map(lambda x : x.hour)

raw_data['weekday'] = raw_data.index.map(lambda x : x.weekday() + 1)

In [3]:
!wget https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip -P data/

--2023-09-07 11:58:02--  https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘data/bike+sharing+dataset.zip’

bike+sharing+datase     [  <=>               ] 273.43K   807KB/s    in 0.3s    

2023-09-07 11:58:02 (807 KB/s) - ‘data/bike+sharing+dataset.zip’ saved [279992]



In [4]:
!unzip data/bike+sharing+dataset.zip -d data/

Archive:  data/bike+sharing+dataset.zip
  inflating: data/Readme.txt         
  inflating: data/day.csv            
  inflating: data/hour.csv           


### Load data

In [5]:
raw_data = pd.read_csv("data/hour.csv", header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [6]:
raw_data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [7]:
# Get weeks number
days = len(raw_data.index.unique())
weeks = days / 7

print(f'days = {days}; weeks = {weeks}')

days = 731; weeks = 104.42857142857143


## Regression Model

### Config

In [8]:
REF_MONTH_START = '2011-01-01'
REF_MONTH_END = '2011-01-28'

CUR_MONTH_START = '2011-01-29'
CUR_MONTH_END = '2011-02-28'

# CUR_WEEK_START = '2011-01-29'
# CUR_WEEK_END = '2011-02-04'
# CUR_WEEK_START = '2011-02-05'
# CUR_WEEK_END = '2011-02-11'
CUR_WEEK_START = '2011-02-12'
CUR_WEEK_END = '2011-02-18'

target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

reports_dir = Path('reports') / f'{CUR_WEEK_START}_{CUR_WEEK_END}'
reports_dir.mkdir(exist_ok=True)

### Model training

In [9]:
reference = raw_data.loc[REF_MONTH_START:REF_MONTH_END]
current = raw_data.loc[CUR_MONTH_START:CUR_MONTH_END]

In [10]:
reference.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [11]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [12]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [13]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [14]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

# Model Monitoring

In [15]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

## Model perfomance

In [16]:
regression_perfomance_report = Report(metrics=[RegressionPreset()])

regression_perfomance_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

/Users/emilykao/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [17]:
model_performance_report_path = reports_dir / 'model_performance.html'
regression_perfomance_report.save_html(model_performance_report_path)

##  Target drift

In [18]:
target_drift_report = Report(metrics=[TargetDriftPreset()])
target_drift_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [19]:
target_drift_report_path = reports_dir / 'target_drift.html'
target_drift_report.save_html(target_drift_report_path)

## Data drift

In [20]:
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features

In [21]:
data_drift_report = Report(metrics=[DataDriftPreset()])
data_drift_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

In [22]:
data_drift_report_path = reports_dir / 'data_drift.html'
data_drift_report.save_html(data_drift_report_path)

## Data quality

In [23]:
column_mapping = ColumnMapping()
column_mapping.numerical_features = numerical_features

In [24]:
data_quality_report = Report(metrics=[DataQualityPreset()])
data_quality_report.run(
    reference_data=reference,
    current_data=current.loc[CUR_WEEK_START:CUR_WEEK_END],
    column_mapping=column_mapping
)

/Users/emilykao/anaconda3/lib/python3.11/site-packages/evidently/metrics/data_integrity/dataset_missing_values_metric.py:165: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [25]:
data_quality_report_path = reports_dir / 'data_quality.html'
data_quality_report.save_html(data_quality_report_path)